In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import *
import random
import sympy as sy
import scipy.linalg as linalg
import scipy
from scipy import integrate
import math

- Usando su número de cédula cómo semilla, obtenga un numero entero aleatorio, n, entre 2 y 20.

In [2]:
random.seed(1007338168)
 

nn = random.randint(2, 20)

nn

2

- Usando el  obtenido, haga una estimación de la siguiente integral hasta dos cifras significativas, usando el método quad del módulo integrate de Scipy

$$\int_0^\infty \sin(x^n)\operatorname{d}x$$

In [3]:
w=integrate.quad(lambda x: np.sin(x**2),0,np.Inf)[0]

w

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  """Entry point for launching an IPython kernel.


-2395.5825842271347

In [4]:
def f(x):
  return np.sin(x**2)

z= integrate.quad(f, 0, np.inf)

res= np.around(z, decimals=2, out=None)

res

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  after removing the cwd from sys.path.


array([-2395.58,  2193.57])

La integral da un numero totalmente incorecto, pues deberia ser 0.626657. sto es evidenciado en el alto error, que puede ser por la naturaleza de la función a integrar y el como la maneja "quad"

In [5]:
def f(x):
  return np.sin(x**2)

z= integrate.quad(f, 0, 6.981)

res= np.around(z, decimals=2, out=None)

res

array([0.62, 0.  ])

Mirando el limite superior, podemos ver que se hace mas preciso, lo que da a pensar que el error tiene que ver con la función en si misma y sus puntos

B) Cargue los datos disponibles en:

https://raw.githubusercontent.com/ComputationalMethods/Evaluacion_2022-1/main/data/lista_pares.json

como un DataFrame de pandas. Este contendrá dos columnas. La columna lista, con una lista de 8 enteros (algunos de ellos repetidos) y una columna pares correspondiente a un diccionario con dos items: un entero rotulado con 'S' y una lista de pares de enteros routulada con 'ψ'. Note que el resultado de la suma en valor absoluto de los dos enteros en cada uno de los pares reproduce el valor del correspondiente 'S'

Seleccione la fila del DataFrame con el índice correspondiente a los dos últimos dígitos de su cédula y desarrolle los siguientes puntos con la correspondiente lista y pares:

In [6]:
from google.colab import drive 
drive.mount("/content/gdrive", force_remount=True)

!cp '/content/gdrive/MyDrive/Colab Notebooks/lista_pares.json' 'lista_pares.json'

Mounted at /content/gdrive


In [7]:
dat= pd.read_json('lista_pares.json')

In [8]:
df= pd.DataFrame.from_dict(dat)
df

,lista,pares
0,"[1, 1, 2, 3, -4, -4, -5, 6]","{'S': 2, 'ψ': [[3, -5], [2, -4], [-4, 6], [1, ..."
1,"[1, 2, 2, 2, -3, -5, -6, 7]","{'S': 4, 'ψ': [[2, 2], [-3, 7], [1, -5], [2, -..."
2,"[1, -2, -2, 4, 5, -7, -7, 8]","{'S': 9, 'ψ': [[4, 5], [1, 8], [-2, -7], [5, 4..."
3,"[1, 2, 2, 4, -5, -5, -7, 8]","{'S': 3, 'ψ': [[4, -7], [-5, 8], [2, -5], [1, ..."
4,"[2, -3, -4, 5, -6, 7, 7, -8]","{'S': 1, 'ψ': [[-6, 7], [7, -8], [5, -6], [2, ..."
...,...,...
240,"[2, -4, -5, 10, 11, -16, -17, 19]","{'S': 21, 'ψ': [[-4, -17], [-5, -16], [10, 11]..."
241,"[5, -7, -7, 11, 13, -17, -17, 19]","{'S': 24, 'ψ': [[5, 19], [-7, -17], [11, 13], ..."
242,"[4, -5, -7, 8, -17, 18, 18, -19]","{'S': 1, 'ψ': [[-7, 8], [4, -5], [-17, 18], [1..."
243,"[2, 2, 5, -7, 10, -14, -17, 19]","{'S': 12, 'ψ': [[2, -14], [2, 10], [-7, 19], [..."


En mis caso seria el 68


In [9]:
#lista

[4,-5,-6,9,9,-12,-13,14]	

#pares

{'S': 18, 'ψ': [[-6, -12], [-5, -13], [4, 14], [9, 9], [-12, -6], [-13, -5], [14, 4]]}

{'S': 18,
 'ψ': [[-6, -12], [-5, -13], [4, 14], [9, 9], [-12, -6], [-13, -5], [14, 4]]}

In [10]:
Datos = df.loc[68,:]
l = Datos["lista"]
p = Datos["pares"]

print(l)
print(p)

[4, -5, -6, 9, 9, -12, -13, 14]
{'S': 18, 'ψ': [[-6, -12], [-5, -13], [4, 14], [9, 9], [-12, -6], [-13, -5], [14, 4]]}


1. Construya una matrix 8x8 simétrica donde cada fila y la correspondiente columna correspondan a un entero de la lista (incluyendo los repetidos); de manera que en los cruces de filas y columnas definidos por los pares de enteros de 'ψ', se genere una entrada aleatoria mayor que cero y menor o igual que 1, con las demas entradas iguales a cero.

In [11]:
m= np.zeros((8,8))
m

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
m2=  ([[0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.]])

for i in range(0,8):
    for j in range(0,8):
        a=np.random.random()
        m2[i][j]=m2[i][j]*a
        m2[j][i]=m2[j][i]*a

sy.Matrix(m2)

Matrix([
[              0.0,               0.0,                0.0,               0.0,                0.0,                0.0,               0.0, 0.273109370138678],
[              0.0,               0.0,                0.0,               0.0,                0.0,                0.0, 0.227122501271139,               0.0],
[              0.0,               0.0,                0.0,               0.0,                0.0, 0.0942422494943858,               0.0,               0.0],
[              0.0,               0.0,                0.0, 0.275369581577006,  0.254493379187432,                0.0,               0.0,               0.0],
[              0.0,               0.0,                0.0, 0.254493379187432, 0.0705280572186874,                0.0,               0.0,               0.0],
[              0.0,               0.0, 0.0942422494943858,               0.0,                0.0,                0.0,               0.0,               0.0],
[              0.0, 0.227122501271139,           

2. Calcule el determinante y establezca explícitamente si es diferente cero (tenga en cuenta la precisión numérica). Recuerde que si el determinante es cero, se espera que al menos uno de los autovalores sea cero

In [13]:
m2

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.27310937013867775],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22712250127113895, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.09424224949438578, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.27536958157700575, 0.25449337918743187, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.25449337918743187, 0.07052805721868735, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.09424224949438578, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.22712250127113895, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.27310937013867775, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [14]:
dt=np.linalg.det(m2)
dt

1.5496012270567885e-06

El determinante es diferente de 0

Organize los autovalores y autovectores de menor a mayor en valor absoluto y compruebe que las matriz de autovectores organizada da lugar a los autovalores en el orden correcto.

In [15]:
W, V= np.linalg.eig(m2)

In [16]:
#autovectores

b = abs(V)

b.sort()

print(b)

[[0.         0.         0.         0.         0.         0.
  0.70710678 0.70710678]
 [0.         0.         0.         0.         0.         0.
  0.70710678 0.70710678]
 [0.         0.         0.         0.         0.         0.
  0.70710678 0.70710678]
 [0.         0.         0.         0.         0.         0.
  0.55975491 0.82865822]
 [0.         0.         0.         0.         0.         0.
  0.55975491 0.82865822]
 [0.         0.         0.         0.         0.         0.
  0.70710678 0.70710678]
 [0.         0.         0.         0.         0.         0.
  0.70710678 0.70710678]
 [0.         0.         0.         0.         0.         0.
  0.70710678 0.70710678]]


In [17]:
#autovalores

b2 = abs(W)

b2.sort()

print(b2)

[0.09424225 0.09424225 0.10138108 0.2271225  0.2271225  0.27310937
 0.27310937 0.44727872]


Establezca explícitamente cuantos autovalores son degenerados en valor absoluto.

Se degeneran 3 autovalores debido al valor absoluto